In [ ]:
# =============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS E CONFIGURAÇÕES INICIAIS
# =============================================================================
# Este bloco importa todas as bibliotecas necessárias e carrega as
# configurações do arquivo configuracoes.conf

import pandas as pd
import json
import configparser
from pathlib import Path
import os

# Bibliotecas Keras/TensorFlow para construção de redes neurais
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, LSTM, Conv1D, Flatten, MaxPooling1D
from keras.optimizers import Adam

# =============================================================================
# CARREGAMENTO DAS CONFIGURAÇÕES
# =============================================================================
# Lê o arquivo de configuração que contém:
# - Caminho dos dados
# - Cidade alvo (Ilhéus ou Itabuna)
# - Tipo de modelo (RNN, LSTM ou CNN)
# - Horizonte de previsão (quantidade de meses)

configs = configparser.ConfigParser()
configs.read("../configuracoes.conf", encoding='utf-8')

data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
objeto = configs["DEFAULT"]["objeto"]        # Ex: "cesta basica"
regiao = configs["DEFAULT"]["regiao"]        # Ex: "Ilheus" ou "Itabuna"
model_name = configs["DEFAULT"]["model"]     # Ex: "RNN", "LSTM" ou "CNN"

# =============================================================================
# PARÂMETROS DO MODELO
# =============================================================================
look_back = 12                                      # Janela temporal: usa 12 meses anteriores
forecast_horizon = int(configs["DEFAULT"]["meses"])  # Quantidade de meses a prever (3, 6 ou 12)

# Previsão preço da cesta básica

## Definir os modelos

In [ ]:
# =============================================================================
# DEFINIÇÃO DAS ARQUITETURAS DE REDES NEURAIS
# =============================================================================

def RNN_model():
    """
    Rede Neural Recorrente (RNN) Simples

    Arquitetura:
    - Camada de entrada: (batch=1, timesteps=12, features=1)
    - Camada RNN: 24 neurônios (captura padrões temporais)
    - Camada densa: forecast_horizon neurônios (saída com previsões)

    Função de perda: MSE (Mean Squared Error)
    Otimizador: Adam com learning rate 0.0003

    Returns:
        Sequential: Modelo RNN compilado
    """
    model = Sequential()
    model.add(Input(batch_shape=(1, look_back, 1)))
    model.add(SimpleRNN(24, stateful=False))
    model.add(Dense(forecast_horizon))
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0003))
    return model

def LSTM_model():
    """
    Rede Neural Long Short-Term Memory (LSTM)

    Arquitetura:
    - Camada de entrada: (batch=1, timesteps=12, features=1)
    - 1ª Camada LSTM: 32 neurônios com return_sequences=True
    - 2ª Camada LSTM: 32 neurônios com stateful=True (mantém estado entre batches)
    - Camada densa: forecast_horizon neurônios (saída com previsões)

    Vantagem: Memória de longo prazo para capturar dependências temporais complexas

    Returns:
        Sequential: Modelo LSTM compilado
    """
    model = Sequential()
    model.add(Input(batch_shape=(1, look_back, 1)))
    model.add(LSTM(32, stateful=False, return_sequences=True))
    model.add(LSTM(32, stateful=True))
    model.add(Dense(forecast_horizon))
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0003))
    return model

def CNN_model():
    """
    Rede Neural Convolucional 1D (CNN)

    Arquitetura:
    - Camada de entrada: (timesteps=12, features=1)
    - Camada Conv1D: 24 filtros, kernel=3, ativação tanh (extrai padrões locais)
    - MaxPooling1D: pool_size=2 (reduz dimensionalidade)
    - Flatten: Achata para camada densa
    - Camada densa intermediária: 15 neurônios com ativação tanh
    - Camada densa de saída: forecast_horizon neurônios

    Vantagem: Captura padrões locais e sazonalidades através de filtros convolucionais

    Returns:
        Sequential: Modelo CNN compilado
    """
    model = Sequential()
    model.add(Input(input_shape=(look_back, 1)))
    model.add(Conv1D(filters=24, kernel_size=3, activation='tanh'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(15, activation='tanh'))
    model.add(Dense(forecast_horizon))
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0003))
    return model

# =============================================================================
# DICIONÁRIO DE MODELOS DISPONÍVEIS
# =============================================================================
# Mapeia o nome do modelo (string) para a função construtora correspondente
models = {
    "RNN": RNN_model,
    "LSTM": LSTM_model,
    "CNN": CNN_model
}

## Prever Cesta

In [ ]:
# =============================================================================
# PREVISÃO DA CESTA BÁSICA
# =============================================================================
# Este bloco realiza todo o pipeline de previsão para a cesta básica:
# 1. Carrega e pré-processa os dados
# 2. Cria sequências temporais
# 3. Treina o modelo selecionado
# 4. Gera previsões
# 5. Salva modelo e resultados

# -----------------------------------------------------------------------------
# 1. CARREGAMENTO E PRÉ-PROCESSAMENTO DOS DADOS
# -----------------------------------------------------------------------------
cesta = pd.read_excel(path)
cesta.drop(['ano'], axis=1, inplace=True)  # Remove coluna 'ano' (não necessária)
cesta = cesta / 1000                        # Normaliza valores (divide por 1000)
cesta = cesta.iloc[::, ::]                 # Seleciona todos os dados

# -----------------------------------------------------------------------------
# 2. CRIAÇÃO DE SEQUÊNCIAS TEMPORAIS (TIME SERIES WINDOWING)
# -----------------------------------------------------------------------------
# Cria colunas adicionais com valores deslocados para formar sequências
# Exemplo: Para look_back=12 e forecast_horizon=3, cria 15 colunas adicionais
# preco t(h+1), preco t(h+2), ..., preco t(h+15)
for n_step in range(1, look_back + forecast_horizon):
    cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values

cesta.dropna(inplace=True)           # Remove linhas com valores NaN (início/fim da série)
cesta.reset_index(drop=True, inplace=True)

# -----------------------------------------------------------------------------
# 3. DIVISÃO DOS DADOS EM TREINO E VALIDAÇÃO
# -----------------------------------------------------------------------------
# X_train: Primeiras 'look_back' colunas (12 meses de histórico)
# y_train: Próximas 'forecast_horizon' colunas (valores a serem previstos)
# X_val: Últimos 12 valores da série (para fazer a previsão final)
X_train = cesta.iloc[::, :look_back:].values
y_train = cesta.iloc[::, look_back::].values
X_val = cesta.iloc[-1::, -12::].values

# -----------------------------------------------------------------------------
# 4. RESHAPE DOS DADOS PARA FORMATO KERAS
# -----------------------------------------------------------------------------
# Formato esperado: (batches, timesteps, features)
# batches: número de amostras
# timesteps: janela temporal (12 meses)
# features: número de variáveis (1 = preço)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

# -----------------------------------------------------------------------------
# 5. CRIAÇÃO E TREINAMENTO DO MODELO
# -----------------------------------------------------------------------------
model = models[model_name]()  # Instancia o modelo selecionado (RNN, LSTM ou CNN)

# Treina o modelo com 150 épocas
# shuffle=False: mantém ordem temporal (importante para séries temporais)
# verbose=0: não exibe progresso detalhado
history = model.fit(X_train, y_train, epochs=150, batch_size=1, shuffle=False, verbose=0)

# -----------------------------------------------------------------------------
# 6. SALVAMENTO DO MODELO TREINADO
# -----------------------------------------------------------------------------
save_dir = Path("../output/models") / regiao
save_dir.mkdir(parents=True, exist_ok=True)
model_file = save_dir / f"{model_name}_{regiao.lower()}_{objeto.lower().replace(' ', '_')}_h{forecast_horizon}.keras"
model.save(model_file)
print(f"✅ Modelo completo salvo em: {model_file}")

# -----------------------------------------------------------------------------
# 7. GERAÇÃO DE PREVISÕES
# -----------------------------------------------------------------------------
# Realiza a previsão para os próximos 'forecast_horizon' meses
forecast = model.predict(X_val, batch_size=1)

# Converte resultados para lista de floats
results = [float(value) for value in forecast[0]]

# -----------------------------------------------------------------------------
# 8. EXPORTAÇÃO DOS RESULTADOS PARA JSON
# -----------------------------------------------------------------------------
objeto = objeto.replace(" ", '_')
output = {objeto.lower(): str(results)}

# Define nome do arquivo baseado no horizonte de previsão
if forecast_horizon == 12:
    file_name = f"previsao_{model_name}_12_meses_{objeto.lower()}_{regiao.lower()}.json"
else:
    file_name = f"previsao_{model_name}_{objeto.lower()}_{regiao.lower()}.json"

full_url = f"../output/previsoes_cesta/{file_name}"
os.makedirs(os.path.dirname(full_url), exist_ok=True)

with open(full_url, "w") as file:
    json.dump(output, file, ensure_ascii=False)

print(f"✅ Arquivo com resultados gerados: {file_name}")
print(f"📊 Previsões para os próximos {forecast_horizon} meses: {[round(r*1000, 2) for r in results]}")

Modelo completo salvo em: ..\output\models\Ilheus\RNN_ilheus_cesta_basica_h3.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Arquivo com resultados gerados


## Prever Produtos

In [ ]:
# =============================================================================
# PREVISÃO DE PRODUTOS INDIVIDUAIS DA CESTA BÁSICA
# =============================================================================
# Este bloco realiza previsões para cada produto individual da cesta básica
# em ambas as cidades (Ilhéus e Itabuna), usando o mesmo pipeline do bloco anterior

path_inicial = "../data/datasets_produtos/"

# Lista de regiões e produtos a serem processados
regioes = ['ilheus', 'itabuna']
produtos = ['acucar', 'arroz', 'banana', 'cafe', 'carne', 'farinha', 'feijao', 'leite',
            'manteiga', 'oleo', 'pao', 'tomate']

print(f"🚀 Iniciando treinamento de {len(regioes)} regiões × {len(produtos)} produtos = {len(regioes) * len(produtos)} modelos\n")

# =============================================================================
# LOOP PRINCIPAL: PROCESSA CADA COMBINAÇÃO REGIÃO × PRODUTO
# =============================================================================
for regiao in regioes:
    print(f"\n{'='*70}")
    print(f"📍 PROCESSANDO REGIÃO: {regiao.upper()}")
    print(f"{'='*70}\n")

    for objeto in produtos:
        print(f"  🔄 Produto: {objeto.capitalize():<12}", end=" → ")

        # ---------------------------------------------------------------------
        # 1. CARREGAMENTO E PRÉ-PROCESSAMENTO DOS DADOS
        # ---------------------------------------------------------------------
        path = f'{path_inicial}{regiao}/{objeto}_{regiao}.xlsx'
        cesta = pd.read_excel(path)
        cesta.drop(['ano'], axis=1, inplace=True)
        cesta = cesta / 1000  # Normalização
        cesta = cesta.iloc[::]

        # ---------------------------------------------------------------------
        # 2. CRIAÇÃO DE SEQUÊNCIAS TEMPORAIS
        # ---------------------------------------------------------------------
        for n_step in range(1, look_back + forecast_horizon):
            cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
        cesta.dropna(inplace=True)
        cesta.reset_index(drop=True, inplace=True)

        # ---------------------------------------------------------------------
        # 3. DIVISÃO E RESHAPE DOS DADOS
        # ---------------------------------------------------------------------
        X_train = cesta.iloc[::, :look_back:].values
        y_train = cesta.iloc[::, look_back::].values
        X_val = cesta.iloc[-1::, -12::].values

        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

        # ---------------------------------------------------------------------
        # 4. CRIAÇÃO E TREINAMENTO DO MODELO
        # ---------------------------------------------------------------------
        model = models[model_name]()
        history = model.fit(X_train, y_train, epochs=150, batch_size=1, shuffle=False, verbose=0)

        # ---------------------------------------------------------------------
        # 5. SALVAMENTO DO MODELO
        # ---------------------------------------------------------------------
        save_dir = Path("../output/models") / regiao / "produtos"
        save_dir.mkdir(parents=True, exist_ok=True)
        model_file = save_dir / f"{model_name}_{regiao.lower()}_{objeto.lower()}_h{forecast_horizon}.keras"
        model.save(model_file)

        # ---------------------------------------------------------------------
        # 6. GERAÇÃO DE PREVISÕES
        # ---------------------------------------------------------------------
        forecast = model.predict(X_val, batch_size=1, verbose=0)
        results = [float(value) for value in forecast[0]]

        # ---------------------------------------------------------------------
        # 7. EXPORTAÇÃO DOS RESULTADOS
        # ---------------------------------------------------------------------
        objeto_formatado = objeto.replace(" ", '_')
        output = {objeto_formatado.lower(): str(results)}

        if forecast_horizon == 12:
            file_name = f"previsao_{model_name}_12_meses_{objeto_formatado.lower()}_{regiao.lower()}.json"
        else:
            file_name = f"previsao_{model_name}_{objeto_formatado.lower()}_{regiao.lower()}.json"

        full_url = f"../output/previsoes_produtos/{regiao}/{file_name}"
        os.makedirs(os.path.dirname(full_url), exist_ok=True)

        with open(full_url, "w") as file:
            json.dump(output, file, ensure_ascii=False)

        print(f"✅ Treinado e salvo")

print(f"\n{'='*70}")
print(f"🎉 PROCESSO CONCLUÍDO! Todos os {len(regioes) * len(produtos)} modelos foram treinados com sucesso.")
print(f"{'='*70}")

Modelo completo salvo em: ..\output\models\ilheus\produtos\RNN_ilheus_acucar_h3.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Arquivo com resultados gerados
Modelo completo salvo em: ..\output\models\ilheus\produtos\RNN_ilheus_arroz_h3.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Arquivo com resultados gerados
Modelo completo salvo em: ..\output\models\ilheus\produtos\RNN_ilheus_banana_h3.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
Arquivo com resultados gerados
Modelo completo salvo em: ..\output\models\ilheus\produtos\RNN_ilheus_cafe_h3.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
Arquivo com resultados gerados
Modelo completo salvo em: ..\output\models\ilheus\produtos\RNN_ilheus_carne_h3.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Arquivo com resultados gerados
Modelo completo salvo em: ..\output\models\ilheus\produtos\RNN_ilheus_farinha_h3.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Arquivo com resultados gerados
Modelo completo salvo em: ..\output\models\ilheus\produtos\RNN_ilhe